In [114]:
import pandas as pd
import os
import sys
import json

# from matplotlib import pyplot as plt

In [115]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [116]:
from helper import calculate_len_and_time,sum_cols

In [117]:
pd.set_option('max_colwidth',None)

In [118]:
def fun(teamid,teams):
    if teamid == teams[0]:
        return teams[1]
    else:
        return teams[0]


In [131]:
df_summary = pd.DataFrame(None,columns=['match_id','index','possession_team','minute','second','player','x','y','endX','endY',
                                            'end_action','end_team','eventendX','eventendY','duration','length','opposition_team'])

# read the file
with open(f'../data/seq/{match_id}.json') as f:
    t_seq = json.load(f)
    
    length = []
    for i in t_seq:
        length.append(i[1]-i[0])
        
    df_all = pd.read_csv(f"../data/20_21/{match_id}.csv")
    
    st_time = [x[0]*60+x[1] for x in df_all.iloc[[x[0] for x in t_seq]][['minute','second']].values]
    end_time = [x[0]*60+x[1] for x in df_all.iloc[[x[1] for x in t_seq]][['minute','second']].values]
    
    duration = []
    for e,s in zip(end_time,st_time):
        duration.append(e-s)
    
    df = pd.DataFrame(None)
    
    df['match_id'] = [match_id]*len(t_seq)
    df['index'] = [x[0] for x in t_seq]
    df[['possession_team','minute','second','player','x','y','endX','endY']] = df_all.iloc[[x[0] for x in t_seq]][['teamId','minute','second','name','x','y','endX','endY']].values
    
    teams = df_all.teamId.unique()
    
    df[['end_action','end_team','eventendX','eventendY']] = df_all.iloc[[x[1] for x in t_seq]][['type_displayName','teamId','x','y']].values
    
    df['duration'] = duration
    
    df['length'] = length
    
    df['opposition_team'] = list(map(lambda teamid:fun(teamid,teams),df_all.iloc[[x[0] for x in t_seq]]['teamId']))
    
    df_summary = df_summary.append(df)

In [132]:
df_summary

,match_id,index,possession_team,minute,second,player,x,y,endX,endY,end_action,end_team,eventendX,eventendY,duration,length,opposition_team
0,104,17,167,0,23.0,Kyle Walker,68.2,0.0,47.2,46.6,KeeperPickup,32,4.6,49.7,30.0,12,32
1,104,39,32,1,25.0,Luke Shaw,36.7,100.0,40.1,94.2,OffsideGiven,32,78.8,95.9,NaN,14,167
2,104,172,32,8,43.0,Luke Shaw,24.9,100.0,26.5,80.8,Interception,167,74.4,21.3,1.0,1,167
3,104,191,167,9,34.0,João Cancelo,10.4,100.0,11.2,85.7,Pass,32,32.2,16.1,18.0,11,32
4,104,260,167,14,37.0,João Cancelo,57.5,100.0,70.8,87.2,BallRecovery,32,6.6,43.6,5.0,2,32
5,104,286,167,16,9.0,Kyle Walker,84.5,0.0,100.0,38.0,Pass,32,4.7,50.1,25.0,1,32
6,104,326,167,17,51.0,João Cancelo,27.9,100.0,21.8,86.6,Foul,167,64.4,80.5,46.0,16,32
7,104,356,167,19,25.0,João Cancelo,37.4,100.0,22.2,90.6,SavedShot,167,90.5,60.6,38.0,12,32
8,104,372,167,20,17.0,João Cancelo,72.2,100.0,61.8,88.2,BallRecovery,32,23.9,70.9,43.0,14,32
9,104,486,167,27,37.0,Kyle Walker,38.0,0.0,25.3,14.1,Pass,167,3.5,41.5,33.0,8,32


In [122]:
t_seq

[[17, 29],
 [39, 53],
 [172, 173],
 [191, 202],
 [260, 262],
 [286, 287],
 [326, 342],
 [356, 368],
 [372, 386],
 [486, 494],
 [495, 500],
 [501, 502],
 [556, 559],
 [597, 604],
 [674, 675],
 [720, 740],
 [741, 751],
 [753, 757],
 [803, 804],
 [810, 813],
 [814, 816],
 [831, 842],
 [885, 889],
 [905, 917],
 [979, 986],
 [1018, 1021],
 [1030, 1032],
 [1078, 1079],
 [1091, 1128],
 [1312, 1314],
 [1319, 1332],
 [1333, 1353],
 [1443, 1449],
 [1462, 1463],
 [1588, 1589]]

In [127]:
df_all.iloc[1588:1590][sum_cols]

,minute,second,teamId,x,y,endX,endY,type_displayName,outcomeType_displayName,name,qualifiers
1588,90,33.0,167,90.3,0.0,80.6,31.0,Pass,Successful,Kyle Walker,"[{'type': {'value': 213, 'displayName': 'Angle'}, 'value': '2.0'}, {'type': {'value': 154, 'displayName': 'IntentionalAssist'}}, {'type': {'value': 141, 'displayName': 'PassEndY'}, 'value': '31.0'}, {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'}, {'type': {'value': 210, 'displayName': 'ShotAssist'}}, {'type': {'value': 140, 'displayName': 'PassEndX'}, 'value': '80.6'}, {'type': {'value': 212, 'displayName': 'Length'}, 'value': '24.3'}, {'type': {'value': 1, 'displayName': 'Longball'}}, {'type': {'value': 107, 'displayName': 'ThrowIn'}}, {'type': {'value': 11113, 'displayName': 'KeyPass'}}]"
1589,90,36.0,167,86.0,32.2,NaN,NaN,SavedShot,Successful,Rodri,"[{'type': {'value': 76, 'displayName': 'LowLeft'}}, {'type': {'value': 55, 'displayName': 'RelatedEventId'}, 'value': '890'}, {'type': {'value': 147, 'displayName': 'BlockedY'}, 'value': '49.1'}, {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'}, {'type': {'value': 72, 'displayName': 'LeftFoot'}}, {'type': {'value': 178, 'displayName': 'StandingSave'}}, {'type': {'value': 233, 'displayName': 'OppositeRelatedEvent'}, 'value': '822'}, {'type': {'value': 146, 'displayName': 'BlockedX'}, 'value': '98.2'}, {'type': {'value': 103, 'displayName': 'GoalMouthZ'}, 'value': '3.2'}, {'type': {'value': 102, 'displayName': 'GoalMouthY'}, 'value': '53.0'}, {'type': {'value': 29, 'displayName': 'Assisted'}}, {'type': {'value': 63, 'displayName': 'BoxRight'}}, {'type': {'value': 154, 'displayName': 'IntentionalAssist'}}, {'type': {'value': 22, 'displayName': 'RegularPlay'}}]"


In [27]:
df_all = pd.read_csv(f"../data/20_21/{match_id}.csv")


In [29]:
teams = df_all.teamId.unique()


In [30]:
teams

array([ 32, 167], dtype=int64)

In [34]:
df_all.iloc[:10]['teamId']

0     32
1    167
2    167
3    167
4    167
5     32
6    167
7    167
8     32
9    167
Name: teamId, dtype: int64

In [36]:
[df_all.iloc[:10]['teamId']]

[32, 167, 167, 167, 167, 32, 167, 167, 32, 167]

[167, 32, 32, 32, 32, 167, 32, 32, 167, 32]

In [56]:
teams

array([ 32, 167], dtype=int64)

In [94]:
from helper import *

In [95]:
sequences(df_all.iloc[172:191],'teamId','outcomeType_value',df_all.iloc[172]['teamId'],'Successful')

190

In [106]:
col_name = 'teamId'
value = 'Successful'

col_name2= 'outcomeType_value'

teamId = df_all.iloc[172]['teamId']

In [109]:
seq =    ~ (df_all.iloc[172:191][col_name].eq(teamId)) & \
    ~ (df_all.iloc[172:191][col_name].shift(-1).eq(teamId)) & \
    ~ (df_all[172:191][col_name].shift(-2).eq(teamId))
    
seq2 = (df_all[col_name2].eq(value)) & \
     (df_all[col_name2].shift(-1).eq(value)) & \
     (df_all[col_name2].shift(-2).eq(value))
    
# seq = seq & seq2
# try:
#     return seq[seq].index.values[0]
# except:
#     return seq.index.values[-1]


In [110]:
seq

172    False
173     True
174     True
175     True
176    False
177    False
178    False
179    False
180    False
181    False
182    False
183    False
184    False
185     True
186     True
187     True
188    False
189    False
190    False
Name: teamId, dtype: bool

In [111]:
seq2

0       False
1       False
2       False
3       False
4       False
        ...  
1604    False
1605    False
1606    False
1607    False
1608    False
Name: outcomeType_value, Length: 1609, dtype: bool

In [113]:
seq[seq & seq2]

Series([], Name: teamId, dtype: bool)